In [1]:
!pip install efficientnet

  Created wheel for efficientnet: filename=efficientnet-0.0.4-cp36-none-any.whl size=14288 sha256=172d44a7bf78511f30b57a9f35e57910daa5865bf70e8eac3770a28680df52f4
  Stored in directory: /root/.cache/pip/wheels/5c/34/68/a611a699a28239e964ccf144c0e767cdb5439fee82ec5de6e0
Successfully built efficientnet


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from efficientnet import EfficientNetB4
from keras import models, layers
import keras

import os
warnings.filterwarnings('ignore')

Using TensorFlow backend.


# Efficient Net  - B4

While scaling individual dimensions improves model performance, we observed that balancing all dimensions of the network—width, depth, and image resolution—against the available resources would best improve overall performance.

Compound Scaling
In the paper, the authors propose a compound scaling method that uses a compound coefficient of
$\phi$ to uniformly scale width, depth, and resolution in a principled way


They propose the following formula

---------------------------------------------------------

depth : $d$ = $\alpha^\phi$

width = $w = \beta^\phi$

resolution = $\gamma = \gamma^\phi$

s.t. $\alpha \cdot \beta^2 \cdot \gamma^2 \approx 2$
where : $\alpha \ge 1, \beta \ge 1, \gamma \ge 1$

----------------------------------------------------
Compound Scaling -- EfficientNet

-----------------------------------------------------


$\phi$ is a user-specified coefficient that controls resources (e.g. Floating Point Operations (FLOPs)) available for model scaling.

$\alpha$, $\beta$, $\gamma$ distribute the resources to depth, width, and resolution respectively.

The FLOPS consumed in a convolutional operation is proportional to $d$, $w^2$, and $r^2$, and this fact is reflected in the above equation. The authors restrict $\alpha \cdot \beta^2 \cdot \gamma^2$ to 2 so that every new $\phi$, the FLOPs needed goes by up $2^{\phi}$

-------------------------------------------------------------

source : https://www.learnopencv.com/efficientnet-theory-code/


about MB Conv layer : https://arxiv.org/pdf/1905.04159.pdf

Starting with EfficientNet-B0, the authors used the following strategy to scale it up.

------------------------------------------------

EfficientNet-B0 : \
224x224 Resolution\
32 Fetures\
9  Layers

------------


Fix $\phi = 1.$

Assume the resource available at any step of scaling is twice of the resource at the previous step.

Do a small grid search over $\alpha$, $\beta$, and $\gamma$ such that the constraint in the above equation is not violated.

The authors found the parameters $\alpha = 1.2, \beta =
1.1, \gamma = 1.15$ to work the best.

Fix $\alpha$, $\beta$, and $\gamma$ as constants and scale up EfficientNet-B0 with different $\phi$ to obtain new scaled networks EfficientNet-B1 to B7.

In [3]:
alpha = 1.2
beta = 1.1
gamma = 1.15

alpha * np.square(beta) * np.square(gamma)

1.92027

In [4]:
df_2015 = pd.read_csv('/content/trainLabels_2015.csv')

FileNotFoundError: ignored

In [0]:
df_2015.head()

In [0]:
os.listdir('/content/train_images_2015/train')[:5]

In [0]:
df_2015['img_path'] = df_2015['image'].apply(lambda x: x + '.jpeg')

In [0]:
df_2015.head()

In [0]:
df_2015.info()

In [0]:
df_2015['level'] = df_2015.level.astype('str')

In [0]:
df_2015.info()

In [0]:
sns.countplot(df_2015['level'])

In [0]:
x = df_2015[['image', 'img_path']]
y = df_2015['level']

In [0]:
print(x.shape)
print(y.shape)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify = y, random_state=2019)

In [0]:
print(x_train.shape)
print(y_train.shape)
print()
print(x_test.shape)
print(y_test.shape)

In [0]:
train = pd.concat([x_train, y_train], axis = 1)
validation = pd.concat([x_test, y_test], axis=1)

In [0]:
print(train.shape)
print(validation.shape)

In [0]:

train_data = image.ImageDataGenerator(rescale = 1./255.,
                                      rotation_range= 360,
                                      width_shift_range = 0.4,
                                      height_shift_range = 0.4,
                                      shear_range = 0.4,
                                      zoom_range = 0.2,
                                      horizontal_flip=True,
                                      vertical_flip=True
                                     )

validation_data = image.ImageDataGenerator(rescale = 1./255.)

In [0]:
display(train.head())
print()
display(validation.head())

In [0]:
img_dir = '/content/train_images_2015/train'

img_size = 380
batch_size = 10


train_generator = train_data.flow_from_dataframe(train, 
                                                 directory = img_dir, 
                                                 x_col = 'img_path', 
                                                 y_col = 'level', 
                                                 target_size = (img_size, img_size),
                                                 color_mode = 'rgb', 
                                                 class_mode = 'categorical',
                                                 batch_size=batch_size, 
                                                 seed = 2019,
                                                 shuffle = False)


validation_generator = validation_data.flow_from_dataframe(validation,
                                                          directory = img_dir,
                                                          x_col = 'img_path',
                                                          y_col = 'level', 
                                                          target_size =(img_size, img_size),
                                                          color_mode = 'rgb', 
                                                          class_mode = 'categorical', 
                                                          batch_size = batch_size,
                                                          seed = 2019,
                                                          shuffle=False)



In [0]:
effcient_model = EfficientNetB4(include_top =False, weights =None)

In [0]:
effcient_model.summary()

## Lambda layers is for theano


```
se_tensor = layers.Conv2D(filters, 1,
                                  activation='sigmoid',
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_expand')(se_tensor)
        if backend.backend() == 'theano':
            # For the Theano backend, we have to explicitly make
            # the excitation weights broadcastable.
            pattern = ([True, True, True, False] if backend.image_data_format() == 'channels_last'
                       else [True, False, True, True])
            se_tensor = layers.Lambda(
                lambda x: backend.pattern_broadcast(x, pattern),
                name=prefix + 'se_broadcast')(se_tensor)
        x = layers.multiply([x, se_tensor], name=prefix + 'se_excite')

```



In [0]:
# lambda_layer = effcient_model.get_layer('lambda_77')
# lambda_layer.get_config()

In [0]:
model = models.Sequential()
model.add(effcient_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.3))
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(5, activation='softmax'))

In [0]:
model.summary()

In [0]:
my_opt = keras.optimizers.nadam(lr = 0.0008, beta_1 = 0.5)

call_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.4, patience=4, min_lr = 1e-14)
call_cp = keras.callbacks.ModelCheckpoint('checkpoint.h5', save_best_only = True)

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=my_opt, metrics=['acc'])

In [0]:
model.fit_generator(train_generator, 
                   steps_per_epoch = (train_generator.n // train_generator.batch_size),
                   epochs = 100, 
                   callbacks = [call_lr, call_cp],
                   validation_data = validation_generator,
                   validation_steps = (validation_generator.n // validation_generator.batch_size))

------------
# Pre-Train with 2015 data
-------------

Next Step : Re-train to 2019 data